# Segmenting and Clustering Neighborhoods in Toronto

by Jianxu Shi   
Data Scientist in Training

*This notebook is created to complete the second peer-graded assignment in course "Applied Data Science Capstone". 
The goal is to segment and cluster the neighborhoods in the city of Toronta. The process consists of five steps.
First, a list of postal codes of Canada, along with associated boroughs and neighborhoods, is downloaded from a website.
Second, the data is pre-processed into a dataframe. Third, coordinates are extracted for each neighborhood using API calls.
Fourth, explore and cluster neighborhoods in Toronto. Last, visualize neighborhoods and clusters on a map.*

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import folium
!pip install lxml

     |████████████████████████████████| 5.8MB 3.1MB/s eta 0:00:01


In [2]:
# step 1: download data from website
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
df = pd.read_html(url)
print(df[0].head(10))

# save table back to csv for future use
file = 'list_of_postal_codes_canada_csv'
df[0].to_csv(file, index=False)

  Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront
5      M5A  Downtown Toronto       Regent Park
6      M6A        North York  Lawrence Heights
7      M6A        North York    Lawrence Manor
8      M7A      Queen's Park      Not assigned
9      M8A      Not assigned      Not assigned


In [3]:
# step 2: process the data into a desired dataframe
df_pc = pd.read_csv(file)

# set the column names, remove Borough='Not assigned', and copy Borough name to Neighborhood='Not assigned'
df_pc.columns = ['PostalCode', 'Borough','Neighborhood']
df_pc = df_pc[df_pc.Borough!='Not assigned']
df_pc.loc[df_pc['Neighborhood'] =='Not assigned' , 'Neighborhood'] = df_pc['Borough']

# merge neighborhoods with the same PostalCode (assuming same Borough as well)
df_merged = df_pc.groupby(['PostalCode','Borough'], sort=False).agg( ', '.join)
df_pc = df_merged.reset_index()

df_pc.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [4]:
# print the shape of dataframe of postalcodes
print(df_pc.shape)

(103, 3)


In [16]:
# step 3: use Geocoder or csv to extract coordinates and add to the dataframe
# !pip install geocoder
import geocoder

# attempt to obtain coordinates through Geocoder unsuccessful, use csv file instead
url_geo = 'http://cocl.us/Geospatial_data'
df_latlon = pd.read_csv(url_geo)

df_latlon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
# clean coordinate data and merge into the previous dataframe to create a new dataframe
df_latlon.columns = ['PostalCode', 'Latitude', 'Longitude']
df_new = pd.merge(df_pc, df_latlon, on='PostalCode')

df_new

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
101,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509
